In [9]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Activation,Dropout,Input
from sklearn import cross_validation



# Read data from store.h5

In [2]:
import pandas as pd
store = pd.HDFStore('store.h5')

# print the item in store

In [3]:
print store.items

<bound method HDFStore.items of <class 'pandas.io.pytables.HDFStore'>
File path: store.h5
/data                   frame        (shape->[1000,9443]) 
/gene_info              frame        (shape->[300,9443])  
/label                  frame        (shape->[1000,1])    
/onehot_data            frame        (shape->[1000,28329])>


In [4]:
data=store['onehot_data']
label=store['label']


# Close the store.h5

In [5]:
store.close()

In [6]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,28319,28320,28321,28322,28323,28324,28325,28326,28327,28328
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


# seperate the data

In [7]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(label)
target=enc.transform(label).toarray()

## use sklearn.cross_validation.train_test_split

In [8]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(data,target, test_size=0.3, random_state=0)

# Use Auto-Encoder

In [ ]:
input_img = Input(shape=(X_train.shape[1],))
encoded = Dense(8000, activation='relu')(input_img)
encoded = Dense(2000, activation='relu')(encoded)
encoded = Dense(500, activation='relu')(encoded)

decoded = Dense(2000, activation='relu')(encoded)
decoded = Dense(8000, activation='relu')(decoded)
decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)

autoencoder = Model(input=input_img, output=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(X_train.values,X_train.values,
                verbose=1,
                nb_epoch=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test.values, X_test.values))

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 21s - loss: 0.6934 - val_loss: 0.6931
Epoch 2/100
700/700 [==============================] - 21s - loss: 0.6929 - val_loss: 0.6926
Epoch 3/100
512/700 [====================>.........] - ETA: 5s - loss: 0.6925 

# Use Keras to build the Nets
[None,28329]->[None,1000]->[None,500]->[None,2]

In [ ]:
model = Sequential()
model.add(Dense(1000, input_dim=X_train.shape[1],weights=[np.zeros((X_train.shape[1],1000)),np.zeros((1000,))]))
model.add(Activation('sigmoid'))
model.add(Dropout(0.7))
model.add(Dense(500,input_dim=1000,weights=[np.zeros((1000,500)),np.zeros((500,))]))
model.add(Activation('sigmoid'))
model.add(Dropout(0.7))
model.add(Dense(2,input_dim=500,weights=[np.zeros((500,2)),np.zeros((2,))]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train.values,y_train,batch_size=100,nb_epoch=50,verbose=1,validation_split=0.2)

In [ ]:

encoding_dim=5000
input_img = Input(shape=(X_train.shape[1],))

encoded = Dense(encoding_dim, activation='relu')(input_img)

decoded = Dense(X_train.shape[1], activation='sigmoid')(encoded)


autoencoder = Model(input=input_img, output=decoded)

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(X_train.values, X_train.values,verbose=1,
                nb_epoch=50,
                batch_size=100,
                shuffle=True,    
                validation_data=(X_test.values,X_test.values))

In [ ]:
ls